<a href="https://colab.research.google.com/github/al2501/fooddesert/blob/main/Figure_1_Choropleth_Map_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Installing our packages

In [ ]:
#Install newest branch
!pip install pysal

#Install the geopandas module
!pip install geopandas


     |████████████████████████████████| 2.4 MB 6.8 MB/s 
     |████████████████████████████████| 95 kB 3.0 MB/s 
     |████████████████████████████████| 60 kB 5.9 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 164 kB 63.1 MB/s 
     |████████████████████████████████| 46 kB 2.7 MB/s 
     |████████████████████████████████| 41 kB 348 kB/s 
     |████████████████████████████████| 209 kB 68.8 MB/s 
     |████████████████████████████████| 5.7 MB 18.7 MB/s 
     |████████████████████████████████| 81 kB 6.5 MB/s 
     |████████████████████████████████| 138 kB 63.4 MB/s 
     |████████████████████████████████| 226 kB 59.5 MB/s 
     |████████████████████████████████| 269 kB 43.2 MB/s 
     |████████████████████████████████| 66.6 MB 98 kB/s 
     |████████████████████████████████| 2.1 MB 41.7 MB/s 
     |████████████████████████████████| 183 kB 48.3 MB/s 
     |████████████████████████████████| 117 kB 48.2 MB/s 
     |██████████████████████

## Setup our enviroment

In [ ]:
!pip install descartes
!pip install mapclassify

In [ ]:
from pysal import *
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pylab
import descartes
import mapclassify

%matplotlib inline

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (20., 16.)

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Importing and cleaning our first dataset: Local Authority Districts 2020 Boundaries. 
The shapefile used for this section was found at geoportal: https://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2020-uk-bfc/explore . 

In [ ]:
#we used a shared google drive file so everybody can have access to the same data and try the code

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# importing and visualisating our shapefile for the first time

data_path = "/content/drive/MyDrive/UCL BASc/Year 2/BASC0005/QM data/third map geofood/Local_Authority_Districts_(December_2020)_UK_BFC.shp"

df1 = gp.read_file(data_path)

df1.head()

In [ ]:
df1['geometry'].head()

In [ ]:
df1.tail()

In [ ]:
#cleaning the dataset and removing unnecessary columns
clean = df1[['LAD20CD','LAD20NM','geometry']]
UK_LTLAs=clean.replace(',', '', regex=True)

In [ ]:
#info for projection
UK_LTLAs.crs

In [ ]:
UK_LTLAs.crs.name
UK_LTLAs.crs.datum

In [ ]:
type(UK_LTLAs.crs)

In [ ]:
#projection using the correct epsg
UK_LTLAs.crs = 'epsg:27700'
target_crs = {'datum':'OSGB36', 'no_defs':True, 'proj':'merc'}
projected_UK_LTLAs = UK_LTLAs.to_crs(crs=target_crs)

In [ ]:
projected_UK_LTLAs['geometry'].head()

In [ ]:
#plotting our boundaries 
figure1 = projected_UK_LTLAs.plot()

## Importing our second dataset: Food deserts dataset. 
Our dataset was found at https://data.cdrc.ac.uk/dataset/e-food-desert-index/resource/efdi-england-and-wales


In [ ]:
#importing dataset 2: food
data_path = "/content/drive/MyDrive/UCL BASc/Year 2/BASC0005/QM data/third map geofood/efdi_england.csv"

food = pd.read_csv(data_path, encoding = 'latin1')
food.head()

In [ ]:
#checking if there is no error
food.replace(',', '', regex=True, inplace=True)
food['Score'] = food['Score'].replace('-', 'NaN', regex=True).astype('float')
food['Score'].head()

In [ ]:
#cleaning the data
food = food.rename(columns={'LSOA or DZ':'LSOA'})
food.drop(labels=34753, axis=0)

In [ ]:
food = food[food.Nation != 'Wales']
food.drop('Rank', axis=1, inplace=True)
food.drop('Nation', axis=1, inplace=True)
food.drop('Decile', axis=1, inplace=True)

##Converting the Lsoas 2011 to LTLA 2020 

File found at : https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-upper-tier-local-authorities-2021-lookup-in-england-and-wales-/explore



In [ ]:
#importing file
data_path = "/content/drive/MyDrive/UCL BASc/Year 2/BASC0005/QM data/first map/loastoward.csv"
convert = pd.read_csv(data_path, encoding = 'latin1')
convert.head()

In [ ]:
# cleaning dataset by removing unecessary columns
convert.drop('LSOA11NM', axis=1, inplace=True)
convert.drop('LAD20NM', axis=1, inplace=True)
convert.drop('WD20CD', axis=1, inplace=True)
convert.drop('WD20NM', axis=1, inplace=True)
convert.drop('ï»¿FID', axis=1, inplace=True)

In [ ]:
convert = convert.rename(columns={'LSOA11CD':'LSOA'})

##Merging our datasets to create geofood

In [ ]:
merge1= convert.merge(food,right_on='LSOA',left_on='LSOA')
merge1.tail()

In [ ]:
merge1.drop(columns=["LSOA"],inplace=True)

In [ ]:
df = pd.DataFrame(merge1, columns = ['LAD20CD', 'Score'])
df

In [ ]:
#visualiasing how many datapoints need to be removed 
new_val = df[df.duplicated('LAD20CD')]
print(new_val)

In [ ]:
#suming the column 'Score' with the mean function for each local authroity

mean1 = df.groupby('LAD20CD').mean({'Score':'first'}).reset_index()
mean1

In [ ]:
#rounding our dataset to 3 significant figures
mean1['Score']=mean1['Score'].round(decimals=3)
mean1['Score'].head()

In [ ]:
geofood = UK_LTLAs.merge(mean1,right_on='LAD20CD',left_on='LAD20CD')

In [ ]:
geofood

In [ ]:
#checking if NUll values are in the dataset 
geofood.isnull().any()

In [ ]:
type(geofood)

### 2 different scales for mapping 

In [ ]:
#projection of our dataset geofood
geofood.crs = 'epsg:27700'
target_crs = {'datum':'OSGB36', 'no_defs':True, 'proj':'merc'}
projected_geofood= geofood.to_crs(crs=target_crs)

In [ ]:
#plotting the map with matplolib
import matplotlib.pyplot as plt
projected_geofood.plot(column='Score',cmap='Blues',legend=True, scheme='fisher_jenks', alpha = 0.5,linewidth=0.8, edgecolor='k')

plt.title('Figure 1- Map representing the distribution of E-food Desert Index (EFDI) across England in 2020 '

In [ ]:
Sources : https://jakevdp.github.io/PythonDataScienceHandbook/04.06-customizing-legends.html